# Data Preprocess

In [ ]:
#!pip install pyradiomics

In [ ]:
#!pip install SimpleITK

In [ ]:
#import os
#import SimpleITK as sitk
#from radiomics import featureextractor

#import os


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
mean = pd.read_csv('/content/drive/MyDrive/data/mean.csv')
print(mean.iloc[:,1])

0     2.820593e+04
1     1.037094e+00
2     3.937125e+01
3     1.681716e+02
4     3.689070e+08
          ...     
90    3.572166e+01
91    2.516745e-03
92    1.959674e+01
93    3.661253e-02
94    1.359508e-01
Name: 105.52314514510368, Length: 95, dtype: float64


## Data upload

In [ ]:
def get_image_mask_pairs(folder):
    """
    Generates pairs of original images and corresponding masks from a folder.
    """
    image_files = {}
    mask_files = {}

    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            if '_mask' in filename:
                # Extract number to match masks and images
                number_masks = filename.split('(')[1].split(')')[0]
                mask_files[number_masks] = os.path.join(folder, filename)

            else:
                number = filename.split('(')[1].split(')')[0]
                image_files[number] = os.path.join(folder, filename)
    #print('number of masks: ', number_masks)
    #print('number of original: ' , number_originals)
    # Pair images with their masks
    pairs = []
    for number, image_path in image_files.items():
        mask_path = mask_files.get(number)
        if mask_path:
            pairs.append((image_path, mask_path))
    return pairs

    #print('number of images: ', number)

def extract_features(image_path, mask_path):
    """
    Extract radiomics features from a given image and mask path.
    """
    # Initialize feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()

    # Load image and mask
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)

    # Extract features
    result = extractor.execute(image, mask)

    return result


malignant_folder = "/content/drive/MyDrive/Colab Health/project/benign_deleted/"
benign_folder = "/content/drive/MyDrive/Colab Health/project/malignant_deleted/"

# Get image-mask pairs for each folder
malignant_pairs = get_image_mask_pairs(malignant_folder)
benign_pairs = get_image_mask_pairs(benign_folder)



'def get_image_mask_pairs(folder):\n    """\n    Generates pairs of original images and corresponding masks from a folder.\n    """\n    image_files = {}\n    mask_files = {}\n\n    for filename in os.listdir(folder):\n        if filename.endswith(\'.png\'):\n            if \'_mask\' in filename:\n                # Extract number to match masks and images\n                number_masks = filename.split(\'(\')[1].split(\')\')[0]\n                mask_files[number_masks] = os.path.join(folder, filename)\n\n            else:\n                number = filename.split(\'(\')[1].split(\')\')[0]\n                image_files[number] = os.path.join(folder, filename)\n    #print(\'number of masks: \', number_masks)\n    #print(\'number of original: \' , number_originals)\n    # Pair images with their masks\n    pairs = []\n    for number, image_path in image_files.items():\n        mask_path = mask_files.get(number)\n        if mask_path:\n            pairs.append((image_path, mask_path))\n    ret

## Extracting Features

In [ ]:
No Need to run this part since we saved the info to csv file


def convert_to_grayscale(image):
    """
    Convert a color image (RGB or RGBA) to grayscale if necessary.
    """
    if image.GetNumberOfComponentsPerPixel() > 1:
        return sitk.VectorMagnitude(image)
    return image

def get_image_mask_pairs(folder):
    """
    Generates pairs of original images and corresponding masks from a folder.
    """
    image_files = {}
    mask_files = {}

    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            number = filename.split('(')[1].split(')')[0]
            if '_mask' in filename:
                mask_files[number] = os.path.join(folder, filename)
            else:
                image_files[number] = os.path.join(folder, filename)

    pairs = []
    for number, image_path in image_files.items():
        mask_path = mask_files.get(number)
        if mask_path:
            pairs.append((image_path, mask_path))

    return pairs

def extract_features(images, masks):
    """
    Extract radiomics features for a list of image and mask paths.
    """
    params = {
        'label': 255,
        'enableCExtensions': True,
        'additionalInfo': True,
        'force2D': True,
        'force2Ddimension': 2
    }
    extractor = featureextractor.RadiomicsFeatureExtractor(**params)

    results = []
    for image_path, mask_path in zip(images, masks):
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)



        # Convert images to grayscale if they are in color
        image = convert_to_grayscale(image)

        # Debug: Check the mask values before extraction
        mask_array = sitk.GetArrayFromImage(mask)
        unique_values = np.unique(mask_array)
        print(f"Unique mask values in {mask_path}: {unique_values}")

        # Extract features
        result = extractor.execute(image, mask)
        results.append(result)

    return results


malignant_pairs = get_image_mask_pairs(malignant_folder)
benign_pairs = get_image_mask_pairs(benign_folder)

# Unzip the pairs to separate lists
malignant_images, malignant_masks = zip(*malignant_pairs)
benign_images, benign_masks = zip(*benign_pairs)

# Extract features for all images
malignant_features = extract_features(malignant_images, malignant_masks)
for features in malignant_features:
    print("Features for a malignant image:", features)

benign_features = extract_features(benign_images, benign_masks)
for features in benign_features:
    print("Features for a benign image:", features)



'\' No Need to run this part since we saved the info to csv file\n\n\ndef convert_to_grayscale(image):\n    """\n    Convert a color image (RGB or RGBA) to grayscale if necessary.\n    """\n    if image.GetNumberOfComponentsPerPixel() > 1:\n        return sitk.VectorMagnitude(image)\n    return image\n\ndef get_image_mask_pairs(folder):\n    """\n    Generates pairs of original images and corresponding masks from a folder.\n    """\n    image_files = {}\n    mask_files = {}\n\n    for filename in os.listdir(folder):\n        if filename.endswith(\'.png\'):\n            number = filename.split(\'(\')[1].split(\')\')[0]\n            if \'_mask\' in filename:\n                mask_files[number] = os.path.join(folder, filename)\n            else:\n                image_files[number] = os.path.join(folder, filename)\n\n    pairs = []\n    for number, image_path in image_files.items():\n        mask_path = mask_files.get(number)\n        if mask_path:\n            pairs.append((image_path, m

In [ ]:
#malignant_features

In [ ]:
#benign_features

In [ ]:
# Creating a DataFrame from the features
malignant_df = pd.DataFrame(malignant_features)
malignant_df['Label'] = 'Malignant'

benign_df = pd.DataFrame(benign_features)
benign_df['Label'] = 'Benign'

# Combining both DataFrames
combined_df = pd.concat([malignant_df, benign_df], ignore_index=True)

# Save DataFrame to CSV
combined_df.to_csv('image_features.csv', index=False)

combined_df.shape

combined_df


"\n# Creating a DataFrame from the features\nmalignant_df = pd.DataFrame(malignant_features)\nmalignant_df['Label'] = 'Malignant'\n\nbenign_df = pd.DataFrame(benign_features)\nbenign_df['Label'] = 'Benign'\n\n# Combining both DataFrames\ncombined_df = pd.concat([malignant_df, benign_df], ignore_index=True)\n\n# Save DataFrame to CSV\ncombined_df.to_csv('image_features.csv', index=False)\n\ncombined_df.shape\n\ncombined_df\n"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd

# **SEGMENTATION**

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, random_split
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image

# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(data_folder) if f.endswith('.png') and not f.endswith('_mask.png')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.data_folder, img_name)
        mask_name = img_name.replace('.png', '_mask.png')
        mask_path = os.path.join(self.data_folder, mask_name)

        image = Image.open(img_path).convert('L')  # Convert image to grayscale
        mask = Image.open(mask_path).convert('L')  # Convert mask to grayscale

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = (mask > 0.5).float()

        return image, mask

# Define the transformations
transform = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
}


# Paths to your images and masks
benign_images_path = '/content/drive/MyDrive/data/benign'
malignant_images_path = '/content/drive/MyDrive/data/malignant'

# Create datasets
benign_dataset = CustomDataset(benign_images_path, transform=transform)
malignant_dataset = CustomDataset(malignant_images_path, transform=transform)

print(benign_dataset.__getitem__(1)[1].unique())

# Combine datasets
combined_dataset = torch.utils.data.ConcatDataset([benign_dataset, malignant_dataset])


# Split dataset into training, validation, and test sets
train_size = int(0.7 * len(combined_dataset))
val_size = int(0.15 * len(combined_dataset))
test_size = len(combined_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(combined_dataset, [train_size, val_size, test_size])

# Update transforms for validation and test sets
train_dataset.dataset.transform = transform['train']
val_dataset.dataset.transform = transform['val']
test_dataset.dataset.transform = transform['test']

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

'''
def load_images_and_masks(images_path):
    images = []
    masks = []
    image_size = (128, 128)  # Adjust based on your needs

def get_image_mask_pairs(folder):
    """
    Generates pairs of original images and corresponding masks from a folder.
    """
    image_files = {}
    mask_files = {}

    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            if '_mask' in filename:
                # Extract number to match masks and images
                number_masks = filename.split('(')[1].split(')')[0]
                mask_files[number_masks] = os.path.join(folder, filename)

            else:
                number = filename.split('(')[1].split(')')[0]
                image_files[number] = os.path.join(folder, filename)
    #print('number of masks: ', number_masks)
    #print('number of original: ' , number_originals)
    # Pair images with their masks
    pairs = []
    for number, image_path in image_files.items():
        mask_path = mask_files.get(number)
        if mask_path:
            pairs.append((image_path, mask_path))
    return pairs

    #print('number of images: ', number)


malignant_folder = "/content/drive/MyDrive/data/project/benign_deleted/"
benign_folder = "/content/drive/MyDrive/Colab Health/project/malignant_deleted/"

# Get image-mask pairs for each folder
malignant_pairs = get_image_mask_pairs(malignant_folder)
benign_pairs = get_image_mask_pairs(benign_folder)




def convert_to_grayscale(image):
    """
    Convert a color image (RGB or RGBA) to grayscale if necessary.
    """
    if image.GetNumberOfComponentsPerPixel() > 1:
        return sitk.VectorMagnitude(image)
    return image

def get_image_mask_pairs(folder):
    """
    Generates pairs of original images and corresponding masks from a folder.
    """
    image_files = {}
    mask_files = {}

    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            number = filename.split('(')[1].split(')')[0]
            if '_mask' in filename:
                mask_files[number] = os.path.join(folder, filename)
            else:
                image_files[number] = os.path.join(folder, filename)

    pairs = []
    for number, image_path in image_files.items():
        mask_path = mask_files.get(number)
        if mask_path:
            pairs.append((image_path, mask_path))

    return pairs




    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    # Ensure masks have the same number of channels as images (required by U-Net)
    masks = np.expand_dims(masks, axis=-1)

    return images, masks

# Load benign and malignant data
benign_images, benign_masks = load_images_and_masks(benign_images_path)
malignant_images, malignant_masks = load_images_and_masks(malignant_images_path)

# Combine benign and malignant data
images = np.concatenate((benign_images, malignant_images), axis=0)
masks = np.concatenate((benign_masks, malignant_masks), axis=0)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

print(f'Training set size: {X_train.shape[0]}')
print(f'Validation set size: {X_val.shape[0]}')

'''


TypeError: 'dict' object is not callable

In [ ]:
def unet_model(input_size=(128, 128, 1)):
    inputs = tf.keras.Input(input_size)

    # Downsampling
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Upsampling
    u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

    return model

model = unet_model()


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Define callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('unet_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=100,
    callbacks=callbacks
)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Predict on a new image
test_image = load_img('path_to_test_image', target_size=image_size, color_mode='grayscale')
test_image = img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict(test_image)
prediction = (prediction > 0.5).astype(np.uint8)  # Thresholding


# **CLASSIFICATION**

#SVM with Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/image_features_uncleaned.csv')

# Prepare features and labels

y = data['Label'].map({'Malignant': 1, 'Benign': 0})
X = data.drop('Label', axis=1)
columns_to_drop = ['diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing',
                   'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
                   'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
                   'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
                   'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Minimum','diagnostics_Image-original_Maximum']

# Drop the specified columns from X
X = X.drop(columns=columns_to_drop)
# Iterate over columns in X
for column in X.columns:
    # Check if any value in the column is of string type
    if X[column].apply(lambda x: isinstance(x, str)).any():
        # Drop the column if it contains strings
        X.drop(column, axis=1, inplace=True)
X_normalized = (X-X.mean()) / X.std()
X_normalized


,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.203916,-0.280594,-0.181076,0.779319,0.312756,-0.159557,0.292484,0.013540,-0.463822,0.199539,...,-0.422414,-0.924868,0.667321,-0.103124,-0.097510,-0.391170,-0.449080,-0.271606,-0.304089,-0.509788
1,0.059099,1.632060,-0.181076,-0.549672,0.270573,0.936820,0.641925,0.667427,-0.447797,0.199539,...,-0.698002,-0.026059,1.497357,-0.744681,-0.024955,2.457518,-0.717432,-0.330754,-0.114923,-0.663132
2,0.523775,-0.024062,-0.181076,-0.264889,0.397123,-0.027328,0.815078,0.398179,-0.484320,0.199539,...,-0.744208,-0.648540,1.282335,-0.753982,-0.089795,-0.055657,-0.497552,-0.342169,-0.104810,-0.527022
3,-0.170372,0.107048,-0.181076,0.241394,0.755681,0.384019,1.007681,0.744355,-0.661440,0.199539,...,0.109656,-1.059575,1.144028,-0.578540,-0.097589,-0.166121,-0.531734,-0.343956,0.040404,-0.605401
4,0.063082,-0.722396,-0.181076,0.937532,0.966598,-0.495268,0.903253,0.744355,-0.681615,0.199539,...,0.598059,0.498662,-0.206678,0.003560,-0.103544,-0.724131,0.849605,-0.215980,0.032235,0.122587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2.070084,-0.790330,-0.181076,3.025946,0.312756,-0.578846,-1.744764,-1.601947,2.740423,0.199539,...,0.428234,-1.766689,-1.339309,-0.096236,-0.099905,-0.743806,0.604503,-0.562867,-0.667383,0.518061
643,1.701700,-0.462072,-0.181076,0.051538,-0.151260,-0.398018,0.433649,0.013540,-0.338430,0.199539,...,-0.085761,-0.180266,-0.125682,0.896150,-0.103715,-0.396675,-0.419013,0.407362,-0.054543,-0.471939
644,-0.656433,-0.837868,-0.181076,-0.644600,-0.235627,-0.698808,0.297421,0.090468,-0.244708,-0.352966,...,-0.333955,-0.450364,-1.210725,2.398882,-0.105150,-0.698219,3.054497,0.069587,0.366863,2.414835
645,1.690425,-0.666339,-0.181076,1.412171,0.565856,-0.452439,0.277047,-0.524956,-0.165918,0.199539,...,0.810024,0.269768,-0.211521,0.707034,-0.103737,-0.661355,-0.057893,0.438716,0.013455,-0.326984


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

import matplotlib.pyplot as plt

# apply SelectKBest
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Perform feature selection
k = 50  # Choose the number of top features you want to select
selector = SelectKBest(score_func=mutual_info_classif, k=k) # try different score_func
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

#Define an SVM classifier
svm_classifier = SVC()

# Define hyperparameters to tune
param_grid = {
    'C': [0.1, 1],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
}

# Perform grid search with cross-validation
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=folds)
grid_search.fit(X_train_selected, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the best model
best_svm_classifier = grid_search.best_estimator_
accuracy = best_svm_classifier.score(X_test_selected, y_test)
print("Accuracy:", accuracy)

# Get the prediction results
y_pred = best_svm_classifier.predict(X_test_selected)

# Calculate precision, recall, and f1 score
print("Classification Report:")
print(classification_report(y_test, y_pred))
from sklearn.metrics import confusion_matrix

# Calculate specificity and sensitivity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Best Hyperparameters: {'C': 1, 'kernel': 'linear'}
Accuracy: 0.8307692307692308
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87        83
           1       0.79      0.72      0.76        47

    accuracy                           0.83       130
   macro avg       0.82      0.81      0.81       130
weighted avg       0.83      0.83      0.83       130

Specificity: 0.891566265060241
Sensitivity: 0.723404255319149
Confusion Matrix:
[[74  9]
 [13 34]]


#SVM with Cleaned Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/image_features_cleaned.csv')

# Prepare features and labels

y = data['Label'].map({'Malignant': 1, 'Benign': 0})
X = data.drop('Label', axis=1)
columns_to_drop = ['diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing',
                   'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
                   'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
                   'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
                   'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Minimum','diagnostics_Image-original_Maximum']

# Drop the specified columns from X
X = X.drop(columns=columns_to_drop)
# Iterate over columns in X
for column in X.columns:
    # Check if any value in the column is of string type
    if X[column].apply(lambda x: isinstance(x, str)).any():
        # Drop the column if it contains strings
        X.drop(column, axis=1, inplace=True)
X_normalized = (X-X.mean()) / X.std()
X_normalized


,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.500921,-0.808355,-0.182702,-0.388981,-0.380692,-0.681164,0.136458,-0.502899,0.132630,0.215393,...,0.924904,0.035137,-1.512136,1.677121,-0.118505,-0.719993,1.538854,0.149736,-0.196354,1.878063
1,-0.116501,4.836150,-0.182702,-0.514145,0.572523,5.136778,1.019384,0.795263,-0.653384,0.215393,...,-0.106721,-1.236449,2.100885,-0.871979,-0.040241,2.669437,-0.769375,-0.498899,-0.053020,-0.757636
2,-0.158499,-0.564573,-0.182702,0.706205,1.267575,-0.066380,0.478570,-0.159268,-0.098052,0.215393,...,1.109031,-0.564680,-0.070079,0.026514,-0.114686,-0.647151,-0.376859,1.061416,0.685695,-0.463820
3,-1.434687,0.245671,-0.182702,-0.420272,-1.770796,-0.404614,-1.213551,-1.457430,2.656374,0.215393,...,-0.328156,1.147078,0.202359,-0.159405,-0.062515,0.765652,-0.700292,0.104573,-0.609771,-0.545762
4,0.986965,0.349953,-0.182702,-0.576727,-0.797723,-0.175291,0.044177,-0.350174,0.234660,0.215393,...,-0.299018,-0.204331,0.726731,-0.697603,-0.056757,0.311976,-0.624010,-0.443317,-0.528320,-0.536116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,-0.549330,-0.502063,-0.182702,1.206862,0.651957,-0.275105,0.343456,-0.350174,-0.377517,0.215393,...,0.171608,-0.284639,0.494159,-0.002641,-0.111921,-0.578510,-0.333622,-0.081187,-0.201476,-0.483401
511,-0.374912,-0.474414,-0.182702,0.174258,1.029271,-0.162708,1.092927,1.033896,-0.676492,0.215393,...,1.190187,-1.176847,-0.304949,-0.573492,-0.113865,-0.559008,-0.235306,-0.067905,0.356573,-0.476777
512,-1.013955,-0.554651,-0.182702,0.612332,-0.003378,-0.421130,0.128280,-0.235630,-0.400166,0.104354,...,-0.550794,-0.751157,0.299693,-0.114514,-0.112872,-0.614001,-0.231499,-0.306116,-0.487261,-0.333781
513,-1.498223,2.759829,-0.182702,-1.139966,-1.731079,-0.124732,-1.571589,-1.037436,1.215521,0.215393,...,-1.976705,1.378760,0.758821,-1.038705,0.791297,3.914300,-0.758969,-0.628719,-0.774387,-0.614414


In [ ]:
# apply SelectKBest
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Perform feature selection
k = 50  # Choose the number of top features you want to select
selector = SelectKBest(score_func=mutual_info_classif, k=k) # try different score_func
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

#Define an SVM classifier
svm_classifier = SVC()

# Define hyperparameters to tune
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
}

# Perform grid search with cross-validation
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=folds)
grid_search.fit(X_train_selected, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the best model
best_svm_classifier = grid_search.best_estimator_
accuracy = best_svm_classifier.score(X_test_selected, y_test)
print("Accuracy:", accuracy)

# Get the prediction results
y_pred = best_svm_classifier.predict(X_test_selected)

# Calculate precision, recall, and f1 score
print("Classification Report:")
print(classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix

# Calculate specificity and sensitivity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

# Print specificity and sensitivity
print("Specificity for cleaned data:", specificity)
print("Sensitivity for cleaned data:", sensitivity)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Best Hyperparameters: {'C': 100, 'kernel': 'linear'}
Accuracy: 0.8058252427184466
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.75      0.68        28
           1       0.90      0.83      0.86        75

    accuracy                           0.81       103
   macro avg       0.76      0.79      0.77       103
weighted avg       0.82      0.81      0.81       103

Specificity for cleaned data: 0.75
Sensitivity for cleaned data: 0.8266666666666667
Confusion Matrix:
[[21  7]
 [13 62]]


# Balancing the Data

In [ ]:
# Separate malignant and benign data
X_malignant = X[y == 1]
y_malignant = y[y == 1]
X_benign = X[y == 0]
y_benign = y[y == 0]
X_malignant

,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,113.220011,2348,1,25.0,147.0,1.914447e+07,2.763054,52.0,4.693916,254.0,...,22.856491,0.039785,5.552298,0.076661,3834.920247,3.958372,0.008192,19.974185,0.027740,0.519816
1,101.750045,191793,1,21.0,195.0,3.014449e+09,3.222402,86.0,2.373652,254.0,...,15.639828,0.018665,8.197166,0.013561,346699.974125,158.783381,0.000170,13.362767,0.033118,0.006499
2,100.969845,10530,1,60.0,230.0,3.356594e+08,2.941040,61.0,4.012957,254.0,...,24.144542,0.029823,6.607938,0.035802,20566.785695,7.285690,0.001535,29.266788,0.060838,0.063721
3,77.261858,37724,1,24.0,77.0,1.615229e+08,2.060702,27.0,12.143853,254.0,...,14.090795,0.058255,6.807373,0.031200,249122.376331,71.820822,0.000411,19.513847,0.012227,0.047763
4,122.249346,41224,1,19.0,126.0,2.795876e+08,2.715044,56.0,4.995102,254.0,...,14.294625,0.035808,7.191233,0.017878,274345.062244,51.097422,0.000676,13.929308,0.015283,0.049641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,119.897044,2423,1,51.0,184.6,3.199774e+07,2.917105,63.0,3.262365,254.0,...,22.476020,0.049831,5.955007,0.082955,3388.768149,3.148177,0.008109,24.522019,0.045909,0.264209
346,74.070028,15841,1,17.0,135.0,1.041522e+08,2.706016,55.0,4.638326,254.0,...,11.287849,0.039213,6.785985,0.018181,110702.899293,20.284385,0.001808,13.428656,0.015328,0.136131
347,98.881147,11136,1,3.0,216.0,1.512306e+08,3.155270,109.0,2.351032,254.0,...,22.195964,0.040459,6.661641,0.071390,3689.514408,39.348266,0.001337,28.165511,0.084056,0.080618
348,129.562188,14179,1,110.0,176.0,2.982883e+08,2.197375,33.0,3.723360,254.0,...,4.365361,0.014350,6.304349,0.009592,180078.691122,15.373552,0.001424,7.238121,0.007088,0.054396


In [ ]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the minority class (malignant data)
X_resampled, y_resampled = smote.fit_resample(X_normalized, y)
X_resampled

,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.500921,-0.808355,-0.182702,-0.388981,-0.380692,-0.681164,0.136458,-0.502899,0.132630,0.215393,...,0.924904,0.035137,-1.512136,1.677121,-0.118505,-0.719993,1.538854,0.149736,-0.196354,1.878063
1,-0.116501,4.836150,-0.182702,-0.514145,0.572523,5.136778,1.019384,0.795263,-0.653384,0.215393,...,-0.106721,-1.236449,2.100885,-0.871979,-0.040241,2.669437,-0.769375,-0.498899,-0.053020,-0.757636
2,-0.158499,-0.564573,-0.182702,0.706205,1.267575,-0.066380,0.478570,-0.159268,-0.098052,0.215393,...,1.109031,-0.564680,-0.070079,0.026514,-0.114686,-0.647151,-0.376859,1.061416,0.685695,-0.463820
3,-1.434687,0.245671,-0.182702,-0.420272,-1.770796,-0.404614,-1.213551,-1.457430,2.656374,0.215393,...,-0.328156,1.147078,0.202359,-0.159405,-0.062515,0.765652,-0.700292,0.104573,-0.609771,-0.545762
4,0.986965,0.349953,-0.182702,-0.576727,-0.797723,-0.175291,0.044177,-0.350174,0.234660,0.215393,...,-0.299018,-0.204331,0.726731,-0.697603,-0.056757,0.311976,-0.624010,-0.443317,-0.528320,-0.536116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,-0.931023,-0.187543,-0.182702,0.222011,0.521042,-0.005503,0.796790,0.352288,-0.573583,0.215393,...,1.075707,-0.205682,-0.017043,-0.242828,-0.109036,-0.411742,-0.432748,-0.192103,-0.037761,-0.560101
696,0.168764,-0.388477,-0.182702,0.564622,0.820919,-0.101541,0.982962,0.834074,-0.653584,0.215393,...,0.149582,-0.621950,0.608047,-0.050140,-0.115965,-0.449846,-0.383751,0.115942,0.344314,-0.570339
697,0.802988,0.402062,-0.182702,0.468042,0.029502,0.321706,0.239548,-0.363320,-0.253671,0.215393,...,-0.939531,-0.885817,1.215903,-0.480149,-0.090992,0.090578,-0.662428,-0.342206,-0.431628,-0.654694
698,-0.576190,1.121745,-0.182702,-1.139966,-0.695845,-0.036353,-0.807597,0.078223,0.132334,0.215393,...,-0.666684,0.477322,0.749314,-1.012763,0.327903,1.624875,-0.658129,-0.873127,-0.551441,-0.513318


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

import matplotlib.pyplot as plt

# apply SelectKBest
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Perform feature selection
k = 50  # Choose the number of top features you want to select
selector = SelectKBest(score_func=mutual_info_classif, k=k) # try different score_func
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

#Define an SVM classifier
svm_classifier = SVC()

# Define hyperparameters to tune
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
}

# Perform grid search with cross-validation
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=folds)
grid_search.fit(X_train_selected, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the best model
best_svm_classifier = grid_search.best_estimator_
accuracy = best_svm_classifier.score(X_test_selected, y_test)
print("Accuracy:", accuracy)

# Get the prediction results
y_pred = best_svm_classifier.predict(X_test_selected)

# Calculate precision, recall, and f1 score
print("Classification Report:")
print(classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix

# Calculate specificity and sensitivity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

# Print specificity and sensitivity
print("Specificity for cleaned data:", specificity)
print("Sensitivity for cleaned data:", sensitivity)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Best Hyperparameters: {'C': 100, 'kernel': 'rbf'}
Accuracy: 0.8428571428571429
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        61
           1       0.91      0.80      0.85        79

    accuracy                           0.84       140
   macro avg       0.84      0.85      0.84       140
weighted avg       0.85      0.84      0.84       140

Specificity for cleaned data: 0.9016393442622951
Sensitivity for cleaned data: 0.7974683544303798
Confusion Matrix:
[[55  6]
 [16 63]]


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50],
    'max_depth': [20],
    'criterion': ['log_loss'],
    'max_features': ['log2']
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier()

# Perform grid search with cross-validation
folds = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=folds, scoring='accuracy', verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_selected, y_train)

##CROS VAL STD
best_params = grid_search.best_params_
rf_best = RandomForestClassifier(**best_params)

# Perform cross-validation and get the scores for each fold
cv_scores = cross_val_score(rf_best, X_train_selected, y_train, cv=5, scoring='accuracy')

# Print the scores for each fold
for i, score in enumerate(cv_scores, 1):
    print(f"Fold {i}: Accuracy = {score}")

# Calculate and print the mean and standard deviation of the cross-validation scores
mean_score = np.mean(cv_scores)
std_dev = np.std(cv_scores)
print(f"Mean accuracy: {mean_score}")
print(f"Standard Deviation: {std_dev}")


## PREDICTION WITH BEST MODEL

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)


# Predictions and evaluation
y_pred = grid_search.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Calculating specificity from the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]
specificity = TN / (TN + FP)
print(conf_matrix)
print(f"Specificity: {specificity}")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fold 1: Accuracy = 0.8839285714285714
Fold 2: Accuracy = 0.9017857142857143
Fold 3: Accuracy = 0.8660714285714286
Fold 4: Accuracy = 0.8482142857142857
Fold 5: Accuracy = 0.8214285714285714
Mean accuracy: 0.8642857142857142
Standard Deviation: 0.027893748842523776
Best parameters found:  {'criterion': 'log_loss', 'max_depth': 20, 'max_features': 'log2', 'n_estimators': 50}
Best accuracy found:  0.8696428571428572
Accuracy: 0.8428571428571429
Precision: 0.8527425422097803
Recall: 0.8428571428571429
[[55  6]
 [16 63]]
Specificity: 0.9016393442622951


# **NN**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Input, concatenate
from tensorflow.keras.models import Model

# Implementing MLP
def mlp_model(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate MLP model
def train_mlp(X_train, y_train, X_test, y_test):
    input_shape = (X_train.shape[1],)
    model = mlp_model(input_shape)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'MLP Model Accuracy: {accuracy}')
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    print(classification_report(y_test, y_pred))


In [ ]:
train_mlp(X_train_selected, y_train, X_test_selected, y_test)